In [2]:
# Pipeline: Copy METAR .tac -> local GIFTs submodule input, build & run encoder, validate IWXXM output
import os, shutil, subprocess, sys, pathlib, datetime
from textwrap import dedent

def find_repo_root(start=None):
    start = pathlib.Path(start or pathlib.Path.cwd()).resolve()
    for p in [start] + list(start.parents):
        if (p / '.git').exists():
            return p
    return start

REPO_ROOT = find_repo_root()
METAR_REPO_DIR = REPO_ROOT  # current repository root
GIFTS_DIR = REPO_ROOT / "GIFTs"  # submodule directory
COMPOSE_FILE = GIFTS_DIR / "docker-compose.yml"
INPUT_DIR = GIFTS_DIR / "input"
OUTPUT_DIR = GIFTS_DIR / "output"
LOG_DIR = GIFTS_DIR / "logs"

if not GIFTS_DIR.exists():
    raise FileNotFoundError(f"GIFTs submodule not found at {GIFTS_DIR}. Initialize with: git submodule update --init --recursive")
if not COMPOSE_FILE.exists():
    raise FileNotFoundError(f"docker-compose file not found: {COMPOSE_FILE}")

# --- Prepare directories ---
for d in (INPUT_DIR, OUTPUT_DIR, LOG_DIR):
    d.mkdir(parents=True, exist_ok=True)

# --- Copy .tac files from this repo data/metar into submodule input ---
source_metar_dir = METAR_REPO_DIR / "data" / "metar"
if not source_metar_dir.exists():
    raise FileNotFoundError(f"Source METAR directory missing: {source_metar_dir}")

# Refresh input directory
for old in INPUT_DIR.glob("*.tac"):
    old.unlink()

copied = 0
for tac in source_metar_dir.glob("*.tac"):
    dest = INPUT_DIR / tac.name
    shutil.copy2(tac, dest)
    copied += 1
print(f"Copied {copied} TAC files into {INPUT_DIR}")

# --- Build images (cached if already built) ---
print("[Docker] Building encoder & validator images (may use cache)...")
subprocess.run(["docker", "compose", "-f", str(COMPOSE_FILE), "build", "gifts-encoder", "gifts-validator"], check=True)
print("[Docker] Build complete.")

# --- Encode TAC -> IWXXM (heuristic introspection) ---
conversion_script = dedent(r'''
import pathlib, sys, inspect, importlib, pkgutil
in_dir = pathlib.Path('/app/input')
out_dir = pathlib.Path('/app/output')
out_dir.mkdir(exist_ok=True)

try:
    import gifts
except ImportError as e:
    print('Import gifts failed:', e); sys.exit(1)

candidates = []
for name in ['METAR', 'metar', 'Metar', 'metar_decoder']:
    try:
        mod = importlib.import_module(f'gifts.{name}')
        candidates.append(mod)
    except Exception:
        pass

if not candidates:
    print('No METAR-like module found under gifts. Available submodules:')
    import pkgutil
    print([m.name for m in pkgutil.iter_modules(gifts.__path__)])
    sys.exit(1)

def attempt_functions(mod, tac_text):
    funcs = [(n,f) for n,f in inspect.getmembers(mod, inspect.isfunction) if not n.startswith('_')]
    funcs.sort(key=lambda nf: sum(k in nf[0].lower() for k in ['tac','xml','encode','convert']), reverse=True)
    for name, fn in funcs:
        try:
            sig = inspect.signature(fn)
            if len(sig.parameters) == 1:
                val = fn(tac_text)
                if isinstance(val, str) and ('<?xml' in val or '<Meteorological' in val or '<iwxxm:' in val):
                    return val, f'{mod.__name__}.{name}'
        except Exception:
            pass
    return None, None

converted = 0
for tac_path in in_dir.glob('*.tac'):
    tac_text = tac_path.read_text(encoding='utf-8', errors='ignore')
    xml_text = None
    source_func = None
    for mod in candidates:
        xml_text, source_func = attempt_functions(mod, tac_text)
        if xml_text:
            break
    if not xml_text:
        print('FAILED to convert (no function produced XML):', tac_path.name)
        continue
    out_file = out_dir / (tac_path.stem + '.xml')
    out_file.write_text(xml_text, encoding='utf-8')
    converted += 1
    print('Converted', tac_path.name, 'using', source_func)

print('Total converted:', converted)
print('Output files now:', [p.name for p in out_dir.glob('*.xml')])
''')

print("[Docker] Running encoder conversion script inside container...")
subprocess.run([
    "docker", "compose", "-f", str(COMPOSE_FILE), "run", "--rm", "gifts-encoder",
    "python", "-c", conversion_script
], check=True)
print("[Encoder] Conversion attempt finished.")

# --- Validate IWXXM outputs ---
print("[Docker] Running IWXXM validation...")
validation_cmd = [
    "docker", "compose", "-f", str(COMPOSE_FILE), "run", "--rm", "gifts-validator",
    "python", "iwxxmValidator.py", "/app/output"
]
validation_result = subprocess.run(validation_cmd, capture_output=True, text=True)
print("[Validator stdout]\n" + validation_result.stdout)
print("[Validator stderr]\n" + validation_result.stderr)
print("Pipeline completed at", datetime.datetime.now())

Copied 34 TAC files into C:\Users\bigme\OneDrive\Documents\GitHub\GIFTs\input
[Docker] Building encoder & validator images (may be cached)...


CalledProcessError: Command '['docker', 'compose', '-f', 'C:\\Users\\bigme\\OneDrive\\Documents\\GitHub\\GIFTs\\docker-compose.yml', 'build', 'gifts-encoder', 'gifts-validator']' returned non-zero exit status 1.

In [5]:
# Direct import test: decode and encode METAR TAC using GIFTs modules
import pathlib, sys, xml.etree.ElementTree as ET


def find_repo_root(start=None):
    start = pathlib.Path(start or pathlib.Path.cwd()).resolve()
    for p in [start] + list(start.parents):
        if (p / '.git').exists():
            return p
    return start

REPO_ROOT = find_repo_root()
GIFTS_DIR = REPO_ROOT / 'GIFTs'
if str(GIFTS_DIR) not in sys.path:
    sys.path.insert(0, str(GIFTS_DIR))

try:
    from gifts import metarDecoder, metarEncoder
except ImportError as e:
    raise RuntimeError(f'Failed to import gifts modules: {e}')

decoder = metarDecoder.Annex3()
encoder = metarEncoder.Annex3()

# Use IWXXM translation examples (latest amendment directory)
source_metar_dir = REPO_ROOT / 'data' / 'iwxxm-translation' / 'Amd79-80-2023' / 'metar'
if not source_metar_dir.exists():
    raise FileNotFoundError(f'METAR examples directory missing: {source_metar_dir}')

output_dir = GIFTS_DIR / 'direct_output'
output_dir.mkdir(parents=True, exist_ok=True)

converted = 0
for tac_path in source_metar_dir.glob('*.tac'):
    tac_text = tac_path.read_text(encoding='utf-8', errors='ignore')
    decoded = decoder(tac_text)
    xml_root = encoder(decoded, tac_text)
    if xml_root is None:
        print('Skip (encoder returned None):', tac_path.name)
        continue
    xml_string = ET.tostring(xml_root, encoding='unicode')
    out_file = output_dir / (tac_path.stem + '.xml')
    out_file.write_text(xml_string, encoding='utf-8')
    converted += 1
    print('Encoded', tac_path.name)
    if converted >= 5:  # limit for quick test
        break

print('Direct import conversion complete. Files encoded this run:', converted)
print('Output dir:', output_dir)
print('Listing:', [p.name for p in output_dir.glob('*.xml')])

Encoded BGBW-282350Z.tac
Encoded BGGH-282350Z.tac
Encoded BGJN-282350Z.tac
Encoded BGTL-290039Z.tac
Encoded BIAR-290000Z.tac
Direct import conversion complete. Files encoded this run: 5
Output dir: C:\Users\bigme\OneDrive\Documents\GitHub\metar-to-IWXXM\GIFTs\direct_output
Listing: ['BGBW-282350Z.xml', 'BGGH-282350Z.xml', 'BGJN-282350Z.xml', 'BGTL-290039Z.xml', 'BIAR-290000Z.xml']
